In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from env import host, user, password

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from graphviz import Graph

In [2]:
def get_titanic_data(host = host, user = user, password = password):
    db = 'titanic_db'
    return pd.read_sql('SELECT * FROM passengers', f'mysql+pymysql://{user}:{password}@{host}/{db}')

In [3]:
titanic = get_titanic_data()

In [4]:
titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [5]:
# What is your baseline prediction?
# Based on the numbers baseline should be that passengers did not survive

titanic.groupby('survived').count()

,passenger_id,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
survived,,,,,,,,,,,,
0,549,549,549,424,549,549,549,549,549,67,549,549
1,342,342,342,290,342,342,342,340,342,136,340,342


In [6]:
titanic['baseline'] = 0

In [7]:
549 / (549 + 342)

0.6161616161616161

In [8]:
titanic.isna().sum()

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
deck            688
embark_town       2
alone             0
baseline          0
dtype: int64

In [9]:
titanic['sex'] = (titanic['sex'] == 'female')

In [10]:
titanic.drop(columns = ['embark_town'], inplace = True)

In [11]:
# Create dummy variables of the species name.
titanic.drop(columns = ['deck'], inplace = True)
titanic.dropna(how = 'any', inplace = True)
dummies_embarked = pd.get_dummies(titanic[['embarked']])
dummies_class = pd.get_dummies(titanic[['class']])
titanic.drop(columns = ['embarked', 'passenger_id', 'class'], inplace = True)


In [12]:
titanic = pd.concat([titanic, dummies_embarked, dummies_class], axis=1)

In [13]:
# What is your baseline accuracy?
# 61.62%
titanic

,survived,pclass,sex,age,sibsp,parch,fare,alone,baseline,embarked_C,embarked_Q,embarked_S,class_First,class_Second,class_Third
0,0,3,False,22.0,1,0,7.2500,0,0,0,0,1,0,0,1
1,1,1,True,38.0,1,0,71.2833,0,0,1,0,0,1,0,0
2,1,3,True,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,1,1,True,35.0,1,0,53.1000,0,0,0,0,1,1,0,0
4,0,3,False,35.0,0,0,8.0500,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,True,39.0,0,5,29.1250,0,0,0,1,0,0,0,1
886,0,2,False,27.0,0,0,13.0000,1,0,0,0,1,0,1,0
887,1,1,True,19.0,0,0,30.0000,1,0,0,0,1,1,0,0
889,1,1,False,26.0,0,0,30.0000,1,0,1,0,0,1,0,0


In [14]:
# Fit the decision tree classifier to your training sample and transform 
# (i.e. make predictions on the training sample)
train, test = train_test_split(titanic, test_size=.2, random_state=123, stratify=titanic['survived'])
train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train['survived'])

In [15]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,alone,baseline,embarked_C,embarked_Q,embarked_S,class_First,class_Second,class_Third
450,0,2,False,36.0,1,2,27.75,0,0,0,0,1,0,1,0
543,1,2,False,32.0,1,0,26.00,0,0,0,0,1,0,1,0
157,0,3,False,30.0,0,0,8.05,1,0,0,0,1,0,0,1
462,0,1,False,47.0,0,0,38.50,1,0,0,0,1,1,0,0
397,0,2,False,46.0,0,0,26.00,1,0,0,0,1,0,1,0


In [85]:
x_train = train.drop(columns = ['survived'])
y_train = train.survived

x_validate = validate.drop(columns = ['survived'])
y_validate = validate.survived

x_test = test.drop(columns = ['survived'])
y_test = test.survived

In [17]:
clf3 = DecisionTreeClassifier(max_depth=3, random_state=123)
clf3 = clf3.fit(x_train, y_train)
y3_pred = clf3.predict(x_train)

In [18]:
dot_data = export_graphviz(clf3, feature_names= x_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('tips_decision_tree', view=True, format="pdf")

'tips_decision_tree.pdf'

In [19]:
# Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, 
# precision, recall, f1-score, and support.


# True positive: survived predicted survived
# True negative: drown predicted drown
# False positive: drown predicted survived
# False negative: survived predicted drown

In [20]:
confusion_matrix(y_train, y3_pred)

array([[230,   7],
       [ 56, 105]])

In [21]:
print(classification_report(y_train, y3_pred))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88       237
           1       0.94      0.65      0.77       161

    accuracy                           0.84       398
   macro avg       0.87      0.81      0.82       398
weighted avg       0.86      0.84      0.83       398



In [22]:
clf4 = DecisionTreeClassifier(max_depth=4, random_state=123)
clf4 = clf4.fit(x_train, y_train)
y4_pred = clf4.predict(x_train)

In [23]:
confusion_matrix(y_train, y4_pred)

array([[231,   6],
       [ 55, 106]])

In [24]:
print(classification_report(y_train, y4_pred))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88       237
           1       0.95      0.66      0.78       161

    accuracy                           0.85       398
   macro avg       0.88      0.82      0.83       398
weighted avg       0.86      0.85      0.84       398



In [25]:
y3_val = clf3.predict(x_validate)
print(classification_report(y_validate, y3_val))

              precision    recall  f1-score   support

           0       0.75      0.95      0.84       102
           1       0.88      0.52      0.65        69

    accuracy                           0.78       171
   macro avg       0.81      0.74      0.75       171
weighted avg       0.80      0.78      0.76       171



In [26]:
y4_val = clf4.predict(x_validate)
print(classification_report(y_validate, y4_val))

              precision    recall  f1-score   support

           0       0.74      0.94      0.83       102
           1       0.86      0.52      0.65        69

    accuracy                           0.77       171
   macro avg       0.80      0.73      0.74       171
weighted avg       0.79      0.77      0.76       171



In [27]:
clf5 = DecisionTreeClassifier(max_depth=5, random_state=123)
clf5 = clf5.fit(x_train, y_train)
y5_pred = clf5.predict(x_train)

In [28]:
print(classification_report(y_train, y5_pred))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89       237
           1       0.89      0.74      0.81       161

    accuracy                           0.86       398
   macro avg       0.87      0.84      0.85       398
weighted avg       0.86      0.86      0.86       398



In [29]:
y5_val = clf5.predict(x_validate)
print(classification_report(y_validate, y5_val))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       102
           1       0.80      0.62      0.70        69

    accuracy                           0.78       171
   macro avg       0.79      0.76      0.77       171
weighted avg       0.79      0.78      0.78       171



In [30]:
dot_data = export_graphviz(clf4, feature_names= x_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('tips_decision_tree', view=True, format="pdf")

'tips_decision_tree.pdf'

In [31]:
# Which model performs better on your in-sample data?
# for in sample data adjusting the max depth is always going to lead to a higher in sample accuracy

In [32]:
# Which model performs best on your out-of-sample data, the validate set?
# A max depth of 5 resulted in the most accurate model when used with the validate data set.

In [34]:
# Fit the Random Forest classifier to your training sample and transform 
# (i.e. make predictions on the training sample) 
# setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.
from sklearn.ensemble import RandomForestClassifier

In [35]:
rf10 = RandomForestClassifier(max_depth = 10, min_samples_leaf = 2, random_state=123)
rf10 = rf10.fit(x_train, y_train)
y10_pred = rf10.predict(x_train)

In [68]:
def gimme_them_weights(rf, x_train):
    feat = rf.feature_importances_
    key = x_train.columns.tolist()
    val = feat.tolist()
    val = [round(num, 2) for num in val]
    res = {key[i]: val[i] for i in range(len(key))} 
    return res

In [69]:
gimme_them_weights(rf10, x_train)

{'pclass': 0.07,
 'sex': 0.26,
 'age': 0.25,
 'sibsp': 0.03,
 'parch': 0.04,
 'fare': 0.21,
 'alone': 0.02,
 'baseline': 0.0,
 'embarked_C': 0.02,
 'embarked_Q': 0.01,
 'embarked_S': 0.01,
 'class_First': 0.03,
 'class_Second': 0.01,
 'class_Third': 0.05}

In [70]:
print(classification_report(y_train, y10_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       237
           1       1.00      0.91      0.95       161

    accuracy                           0.96       398
   macro avg       0.97      0.96      0.96       398
weighted avg       0.97      0.96      0.96       398



In [71]:
confusion_matrix(y_train, y10_pred)

array([[237,   0],
       [ 14, 147]])

In [77]:
rf6 = RandomForestClassifier(max_depth = 6, min_samples_leaf = 2, random_state=123)
rf6 = rf6.fit(x_train, y_train)
y6_pred = rf6.predict(x_train)

In [78]:
gimme_them_weights(rf6, x_train)

{'pclass': 0.1,
 'sex': 0.35,
 'age': 0.14,
 'sibsp': 0.03,
 'parch': 0.03,
 'fare': 0.16,
 'alone': 0.02,
 'baseline': 0.0,
 'embarked_C': 0.03,
 'embarked_Q': 0.0,
 'embarked_S': 0.02,
 'class_First': 0.04,
 'class_Second': 0.02,
 'class_Third': 0.07}

In [79]:
print(classification_report(y_train, y6_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       237
           1       0.92      0.76      0.83       161

    accuracy                           0.88       398
   macro avg       0.89      0.86      0.87       398
weighted avg       0.88      0.88      0.87       398



In [80]:
confusion_matrix(y_train, y6_pred)

array([[227,  10],
       [ 39, 122]])

In [ ]:
# Which model performs better on your in-sample data?
# The model with max depth of 10 performed better on in sample data than the model with a max depth of 6

In [86]:
y10_val = rf10.predict(x_validate)
print(classification_report(y_validate, y10_val))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       102
           1       0.76      0.64      0.69        69

    accuracy                           0.77       171
   macro avg       0.77      0.75      0.76       171
weighted avg       0.77      0.77      0.77       171



In [87]:
y6_val = rf6.predict(x_validate)
print(classification_report(y_validate, y6_val))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       102
           1       0.85      0.58      0.69        69

    accuracy                           0.79       171
   macro avg       0.81      0.76      0.77       171
weighted avg       0.80      0.79      0.78       171



In [ ]:
# Which model performs best on your out-of-sample data, the validate set?
# The model with a max depth of 6 performed better than the model with a max depth of 10